# R による経済分析サンプル

このノートブックでは、Rを使った基本的な経済分析の例を示します。
IRkernelを使用してJupyter環境でRコードを実行します。

In [ ]:
# 必要なライブラリの読み込み
library(tidyverse)
library(ggplot2)
library(dplyr)
library(broom)
library(stargazer)

# 環境確認
R.version.string
packageVersion("tidyverse")

## 1. サンプルデータの生成

In [ ]:
# 経済データをシミュレーション
set.seed(42)
n <- 1000

# 説明変数
education <- rnorm(n, mean = 12, sd = 3)  # 教育年数
experience <- runif(n, min = 0, max = 40)  # 経験年数
age <- education + experience + rnorm(n, mean = 6, sd = 2)

# 被説明変数（対数賃金）
log_wage <- 2.5 + 0.1 * education + 0.05 * experience - 0.001 * experience^2 + rnorm(n, mean = 0, sd = 0.3)
wage <- exp(log_wage)

# データフレームの作成
econ_data <- data.frame(
  wage = wage,
  log_wage = log_wage,
  education = education,
  experience = experience,
  age = age
)

# データの確認
head(econ_data)
summary(econ_data)

## 2. 記述統計とデータ可視化

In [ ]:
# 記述統計
econ_data %>%
  select(wage, education, experience, age) %>%
  summary()

# 相関行列
cor(econ_data[, c("log_wage", "education", "experience", "age")])

In [ ]:
# 賃金分布のヒストグラム
p1 <- ggplot(econ_data, aes(x = wage)) +
  geom_histogram(bins = 30, fill = "skyblue", alpha = 0.7) +
  labs(title = "賃金分布", x = "賃金", y = "頻度") +
  theme_minimal()

print(p1)

In [ ]:
# 対数賃金分布のヒストグラム
p2 <- ggplot(econ_data, aes(x = log_wage)) +
  geom_histogram(bins = 30, fill = "lightgreen", alpha = 0.7) +
  labs(title = "対数賃金分布", x = "対数賃金", y = "頻度") +
  theme_minimal()

print(p2)

In [ ]:
# 教育と賃金の散布図
p3 <- ggplot(econ_data, aes(x = education, y = log_wage)) +
  geom_point(alpha = 0.6) +
  geom_smooth(method = "lm", color = "red") +
  labs(title = "教育年数と対数賃金の関係", x = "教育年数", y = "対数賃金") +
  theme_minimal()

print(p3)

## 3. 回帰分析（Mincer型賃金関数）

In [ ]:
# 基本的なMincer型賃金関数の推定
# log(wage) = β0 + β1*education + β2*experience + β3*experience^2 + ε

# 経験年数の2乗項を追加
econ_data$experience_sq <- econ_data$experience^2

# 単回帰：教育のみ
model1 <- lm(log_wage ~ education, data = econ_data)

# 重回帰：教育 + 経験
model2 <- lm(log_wage ~ education + experience, data = econ_data)

# 完全なMincer型：教育 + 経験 + 経験^2
model3 <- lm(log_wage ~ education + experience + experience_sq, data = econ_data)

# 結果の表示
summary(model1)
summary(model2) 
summary(model3)

In [ ]:
# broomを使った整理された結果
tidy_results <- list(
  model1 = tidy(model1),
  model2 = tidy(model2),
  model3 = tidy(model3)
)

print("=== Model 1: Education only ===")
print(tidy_results$model1)

print("\n=== Model 2: Education + Experience ===")
print(tidy_results$model2)

print("\n=== Model 3: Full Mincer equation ===")
print(tidy_results$model3)

In [ ]:
# モデル比較
model_comparison <- data.frame(
  Model = c("Education only", "Education + Experience", "Full Mincer"),
  R_squared = c(summary(model1)$r.squared, summary(model2)$r.squared, summary(model3)$r.squared),
  Adj_R_squared = c(summary(model1)$adj.r.squared, summary(model2)$adj.r.squared, summary(model3)$adj.r.squared),
  AIC = c(AIC(model1), AIC(model2), AIC(model3))
)

print(model_comparison)

## 4. 診断とロバストネスチェック

In [ ]:
# 残差プロット
econ_data$residuals <- residuals(model3)
econ_data$fitted <- fitted(model3)

# 残差vs予測値プロット
p4 <- ggplot(econ_data, aes(x = fitted, y = residuals)) +
  geom_point(alpha = 0.6) +
  geom_hline(yintercept = 0, color = "red", linetype = "dashed") +
  labs(title = "残差 vs 予測値", x = "予測値", y = "残差") +
  theme_minimal()

print(p4)

In [ ]:
# Q-Qプロット（正規性の確認）
p5 <- ggplot(econ_data, aes(sample = residuals)) +
  stat_qq() +
  stat_qq_line(color = "red") +
  labs(title = "Q-Qプロット（残差の正規性）") +
  theme_minimal()

print(p5)

## 5. 経済的解釈

In [ ]:
# 教育の収益率の計算
education_coef <- coef(model3)["education"]
cat("教育の収益率（1年当たり）:", round(education_coef * 100, 2), "%\n")

# 経験の収益率（年数による変化）
exp_coef <- coef(model3)["experience"]
exp_sq_coef <- coef(model3)["experience_sq"]

# 経験年数別の限界収益率
experience_levels <- c(0, 5, 10, 15, 20, 25, 30)
marginal_returns <- exp_coef + 2 * exp_sq_coef * experience_levels

returns_df <- data.frame(
  Experience = experience_levels,
  Marginal_Return_Percent = round(marginal_returns * 100, 3)
)

print("経験年数別の限界収益率:")
print(returns_df)

# 経験の収益率が最大になる年数
optimal_experience <- -exp_coef / (2 * exp_sq_coef)
cat("\n経験の収益率が最大になる年数:", round(optimal_experience, 1), "年\n")

## まとめ

このノートブックでは、Rを使用して以下の分析を実行しました：

1. **データ生成**: 教育、経験、賃金の関係をシミュレート
2. **記述統計**: データの基本的な特性を確認
3. **可視化**: ヒストグラム、散布図による関係の可視化
4. **回帰分析**: Mincer型賃金関数の推定
5. **診断**: 残差分析による仮定の検証
6. **解釈**: 経済的な含意の計算

### 主な結果
- 教育1年当たりの収益率: 約10%
- 経験の収益率は逓減的（経験^2の係数が負）
- モデルは良好な当てはまりを示している

次のステップとして、実際のデータを使用した分析や、より高度な計量経済学的手法（操作変数法、パネルデータ分析など）に進むことができます。